In [2]:
import copy
import os.path
import sys

In [8]:
os.getcwd()

'/home/minwoo/LESN/DecomposeBERT/experiment'

In [9]:
sys.path.append('/home/minwoo/LESN/DecomposeBERT/')

In [10]:
from utils.model_utils.evaluate import evaluate_model, test_f1, test

from utils.model_utils.load_model import *
from utils.model_utils.model_config import ModelConfig
from utils.dataset_utils.load_dataset import load_dataset
from utils.decompose_utils.weight_remover import WeightRemoverBert
from utils.decompose_utils.concern_identification import ConcernIdentificationBert
from utils.decompose_utils.tangling_identification import TanglingIdentification
from transformers import AutoConfig
from tqdm import tqdm
from datetime import datetime
from pprint import pp
from utils.decompose_utils.sampling import sampling_class
import torch

2024-04-09 15:40:17.451614: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/minwoo/anaconda3/envs/DecomposeBERT/lib/python3.8/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [11]:
# model_name = "prajjwal1/bert-tiny"
# model_dir = "tiny bert"
model_name = "bert-base-uncased"
model_dir = "bert"
data = "SDG"
num_labels = 16
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
checkpoint_name = "best_model.pt"
config = AutoConfig.from_pretrained(model_name, num_labels=num_labels)
model_config = ModelConfig(
    _model_name=model_name,
    _model_dir=model_dir,
    _data=data,
    _is_pretrained=True,
    _transformer_config=config,
    _checkpoint_name=checkpoint_name,
    _device=device,
)
model, tokenizer, checkpoint = load_classification_model(model_config)

train_dataloader, valid_dataloader, test_dataloader = load_dataset(
    model_config, tokenizer, batch_size=32, test_size=0.3
)
# print("Start Time:" + datetime.now().strftime("%H:%M:%S"))
# evaluate_model(model, model_config, test_dataloader)

Loading dataset
Loading preprocessed data...
Loading preprocessed data...
Loading preprocessed data...


In [12]:
for k in range(3):
    i = 4
    print("#Module " + str(i) + " in progress....")
    num_samples = 64

    positive_samples1 = sampling_class(
        train_dataloader, i, 20, num_labels, True, 4, device=device
    )
    positive_samples2 = sampling_class(
        train_dataloader, i, num_samples, num_labels, True, 4, device=device
    )
    negative_samples = sampling_class(
        train_dataloader, i, num_samples, num_labels, False, 4, device=device
    )

    module1 = copy.deepcopy(model)
    w = WeightRemoverBert(model, p=0.9)
    ci1 = ConcernIdentificationBert(model, p=0.6)
    ti1 = TanglingIdentification(model, p=0.9)

    ff1 = [
        [
            torch.sum(
                model.bert.encoder.layer[num].intermediate.dense.weight != 0
            ).item()
        ]
        for num in range(config.num_hidden_layers)
    ]
    ff2 = [
        [torch.sum(model.bert.encoder.layer[num].output.dense.weight != 0).item()]
        for num in range(config.num_hidden_layers)
    ]
    pooler = [torch.sum(model.bert.pooler.dense.weight != 0).item()]
    print("origin")
    j = 0
    print(j)
    results = test_f1(model, test_dataloader, model_config, False)
    pp(results["details"][f"{i}"])

    print("Start Positive CI sparse")

    for batch in positive_samples1:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t1 = w.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    print("Start Positive CI after sparse")

    for batch in positive_samples2:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t1 = ci1.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    print("Start Negative TI")

    for batch in negative_samples:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t = ti1.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    for num in range(config.num_hidden_layers):
        print(f"f({ff1[num]})")
        print("\n")
    for num in range(config.num_hidden_layers):
        print(f"f({ff2[num]})")
        print("\n")
    print(f"f({pooler})")
    print("\n")

#Module 4 in progress....
origin
0
{'precision': 0.8224719101123595,
 'recall': 0.8433179723502304,
 'f1-score': 0.832764505119454,
 'support': 434.0}
Start Positive CI sparse
1
{'precision': 0.8317307692307693,
 'recall': 0.7972350230414746,
 'f1-score': 0.8141176470588236,
 'support': 434.0}
2
{'precision': 0.8390243902439024,
 'recall': 0.7926267281105991,
 'f1-score': 0.8151658767772512,
 'support': 434.0}
3
{'precision': 0.8493827160493828,
 'recall': 0.7926267281105991,
 'f1-score': 0.8200238379022646,
 'support': 434.0}
4
{'precision': 0.8511166253101737,
 'recall': 0.7903225806451613,
 'f1-score': 0.8195937873357229,
 'support': 434.0}
5
{'precision': 0.8528678304239401,
 'recall': 0.7880184331797235,
 'f1-score': 0.8191616766467066,
 'support': 434.0}
Start Positive CI after sparse
6
{'precision': 0.8484107579462102,
 'recall': 0.7995391705069125,
 'f1-score': 0.8232502965599051,
 'support': 434.0}
7
{'precision': 0.7645788336933045,
 'recall': 0.815668202764977,
 'f1-score': 

In [13]:
for k in range(3):
    i = 4
    print("#Module " + str(i) + " in progress....")
    num_samples = 64

    positive_samples1 = sampling_class(
        train_dataloader, i, 20, num_labels, True, 4, device=device
    )
    positive_samples2 = sampling_class(
        train_dataloader, i, num_samples, num_labels, True, 4, device=device
    )
    negative_samples = sampling_class(
        train_dataloader, i, num_samples, num_labels, False, 4, device=device
    )

    module1 = copy.deepcopy(model)
    w = WeightRemoverBert(model, p=0.9)
    ci1 = ConcernIdentificationBert(model, p=0.6)
    ti1 = TanglingIdentification(model, p=0.9)

    ff1 = [
        [
            torch.sum(
                model.bert.encoder.layer[num].intermediate.dense.weight != 0
            ).item()
        ]
        for num in range(config.num_hidden_layers)
    ]
    ff2 = [
        [torch.sum(model.bert.encoder.layer[num].output.dense.weight != 0).item()]
        for num in range(config.num_hidden_layers)
    ]
    pooler = [torch.sum(model.bert.pooler.dense.weight != 0).item()]
    print("origin")
    j = 0
    print(j)
    results = test_f1(model, test_dataloader, model_config, False)
    pp(results["details"][f"{i}"])

    print("Start Positive CI sparse")

    for batch in positive_samples1:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t1 = w.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    print("Start Positive CI after sparse")

    for batch in positive_samples2:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t1 = ci1.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    print("Start Negative TI")

    for batch in negative_samples:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t = ti1.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    for num in range(config.num_hidden_layers):
        print(f"f({ff1[num]})")
        print("\n")
    for num in range(config.num_hidden_layers):
        print(f"f({ff2[num]})")
        print("\n")
    print(f"f({pooler})")
    print("\n")

#Module 4 in progress....
origin
0
{'precision': 0.8224719101123595,
 'recall': 0.8433179723502304,
 'f1-score': 0.832764505119454,
 'support': 434.0}
Start Positive CI sparse
1
{'precision': 0.825287356321839,
 'recall': 0.8271889400921659,
 'f1-score': 0.8262370540851554,
 'support': 434.0}
2
{'precision': 0.8424821002386634,
 'recall': 0.8133640552995391,
 'f1-score': 0.8276670574443141,
 'support': 434.0}
3
{'precision': 0.838479809976247,
 'recall': 0.8133640552995391,
 'f1-score': 0.8257309941520469,
 'support': 434.0}
4
{'precision': 0.838479809976247,
 'recall': 0.8133640552995391,
 'f1-score': 0.8257309941520469,
 'support': 434.0}
5
{'precision': 0.8461538461538461,
 'recall': 0.8110599078341014,
 'f1-score': 0.8282352941176471,
 'support': 434.0}
Start Positive CI after sparse
6
{'precision': 0.8536585365853658,
 'recall': 0.8064516129032258,
 'f1-score': 0.8293838862559241,
 'support': 434.0}
7
{'precision': 0.7720430107526882,
 'recall': 0.8271889400921659,
 'f1-score': 0.

In [14]:
for k in range(3):
    i = 5
    print("#Module " + str(i) + " in progress....")
    num_samples = 64

    positive_samples1 = sampling_class(
        train_dataloader, i, 20, num_labels, True, 4, device=device
    )
    positive_samples2 = sampling_class(
        train_dataloader, i, num_samples, num_labels, True, 4, device=device
    )
    negative_samples = sampling_class(
        train_dataloader, i, num_samples, num_labels, False, 4, device=device
    )

    module1 = copy.deepcopy(model)
    w = WeightRemoverBert(model, p=0.9)
    ci1 = ConcernIdentificationBert(model, p=0.6)
    ti1 = TanglingIdentification(model, p=0.9)

    ff1 = [
        [
            torch.sum(
                model.bert.encoder.layer[num].intermediate.dense.weight != 0
            ).item()
        ]
        for num in range(config.num_hidden_layers)
    ]
    ff2 = [
        [torch.sum(model.bert.encoder.layer[num].output.dense.weight != 0).item()]
        for num in range(config.num_hidden_layers)
    ]
    pooler = [torch.sum(model.bert.pooler.dense.weight != 0).item()]
    print("origin")
    j = 0
    print(j)
    results = test_f1(model, test_dataloader, model_config, False)
    pp(results["details"][f"{i}"])

    print("Start Positive CI sparse")

    for batch in positive_samples1:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t1 = w.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    print("Start Positive CI after sparse")

    for batch in positive_samples2:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t1 = ci1.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    print("Start Negative TI")

    for batch in negative_samples:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t = ti1.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    for num in range(config.num_hidden_layers):
        print(f"f({ff1[num]})")
        print("\n")
    for num in range(config.num_hidden_layers):
        print(f"f({ff2[num]})")
        print("\n")
    print(f"f({pooler})")
    print("\n")

#Module 5 in progress....
origin
0
{'precision': 0.7239263803680982,
 'recall': 0.8368794326241135,
 'f1-score': 0.7763157894736842,
 'support': 282.0}
Start Positive CI sparse
1
{'precision': 0.7272727272727273,
 'recall': 0.8226950354609929,
 'f1-score': 0.7720465890183027,
 'support': 282.0}
2
{'precision': 0.7142857142857143,
 'recall': 0.8333333333333334,
 'f1-score': 0.7692307692307692,
 'support': 282.0}
3
{'precision': 0.7078313253012049,
 'recall': 0.8333333333333334,
 'f1-score': 0.7654723127035831,
 'support': 282.0}
4
{'precision': 0.7027027027027027,
 'recall': 0.8297872340425532,
 'f1-score': 0.7609756097560975,
 'support': 282.0}
5
{'precision': 0.7053571428571429,
 'recall': 0.8404255319148937,
 'f1-score': 0.7669902912621359,
 'support': 282.0}
Start Positive CI after sparse
6
{'precision': 0.6220472440944882,
 'recall': 0.8404255319148937,
 'f1-score': 0.7149321266968327,
 'support': 282.0}
7
{'precision': 0.6920821114369502,
 'recall': 0.8368794326241135,
 'f1-score'

In [15]:
for k in range(3):
    i = 6
    print("#Module " + str(i) + " in progress....")
    num_samples = 64

    positive_samples1 = sampling_class(
        train_dataloader, i, 20, num_labels, True, 4, device=device
    )
    positive_samples2 = sampling_class(
        train_dataloader, i, num_samples, num_labels, True, 4, device=device
    )
    negative_samples = sampling_class(
        train_dataloader, i, num_samples, num_labels, False, 4, device=device
    )

    module1 = copy.deepcopy(model)
    w = WeightRemoverBert(model, p=0.9)
    ci1 = ConcernIdentificationBert(model, p=0.6)
    ti1 = TanglingIdentification(model, p=0.9)

    ff1 = [
        [
            torch.sum(
                model.bert.encoder.layer[num].intermediate.dense.weight != 0
            ).item()
        ]
        for num in range(config.num_hidden_layers)
    ]
    ff2 = [
        [torch.sum(model.bert.encoder.layer[num].output.dense.weight != 0).item()]
        for num in range(config.num_hidden_layers)
    ]
    pooler = [torch.sum(model.bert.pooler.dense.weight != 0).item()]
    print("origin")
    j = 0
    print(j)
    results = test_f1(model, test_dataloader, model_config, False)
    pp(results["details"][f"{i}"])

    print("Start Positive CI sparse")

    for batch in positive_samples1:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t1 = w.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    print("Start Positive CI after sparse")

    for batch in positive_samples2:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t1 = ci1.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    print("Start Negative TI")

    for batch in negative_samples:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t = ti1.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    for num in range(config.num_hidden_layers):
        print(f"f({ff1[num]})")
        print("\n")
    for num in range(config.num_hidden_layers):
        print(f"f({ff2[num]})")
        print("\n")
    print(f"f({pooler})")
    print("\n")

#Module 6 in progress....
origin
0
{'precision': 0.8515901060070671,
 'recall': 0.7901639344262295,
 'f1-score': 0.8197278911564626,
 'support': 305.0}
Start Positive CI sparse
1
{'precision': 0.8436363636363636,
 'recall': 0.760655737704918,
 'f1-score': 0.8,
 'support': 305.0}
2
{'precision': 0.8481481481481481,
 'recall': 0.7508196721311475,
 'f1-score': 0.7965217391304348,
 'support': 305.0}
3
{'precision': 0.8487084870848709,
 'recall': 0.7540983606557377,
 'f1-score': 0.7986111111111112,
 'support': 305.0}
4
{'precision': 0.8487084870848709,
 'recall': 0.7540983606557377,
 'f1-score': 0.7986111111111112,
 'support': 305.0}
5
{'precision': 0.8582089552238806,
 'recall': 0.7540983606557377,
 'f1-score': 0.8027923211169283,
 'support': 305.0}
Start Positive CI after sparse
6
{'precision': 0.8169491525423729,
 'recall': 0.7901639344262295,
 'f1-score': 0.8033333333333333,
 'support': 305.0}
7
{'precision': 0.8494623655913979,
 'recall': 0.7770491803278688,
 'f1-score': 0.811643835616

In [16]:
for k in range(3):
    i = 7
    print("#Module " + str(i) + " in progress....")
    num_samples = 64

    positive_samples1 = sampling_class(
        train_dataloader, i, 20, num_labels, True, 4, device=device
    )
    positive_samples2 = sampling_class(
        train_dataloader, i, num_samples, num_labels, True, 4, device=device
    )
    negative_samples = sampling_class(
        train_dataloader, i, num_samples, num_labels, False, 4, device=device
    )

    module1 = copy.deepcopy(model)
    w = WeightRemoverBert(model, p=0.9)
    ci1 = ConcernIdentificationBert(model, p=0.6)
    ti1 = TanglingIdentification(model, p=0.9)

    ff1 = [
        [
            torch.sum(
                model.bert.encoder.layer[num].intermediate.dense.weight != 0
            ).item()
        ]
        for num in range(config.num_hidden_layers)
    ]
    ff2 = [
        [torch.sum(model.bert.encoder.layer[num].output.dense.weight != 0).item()]
        for num in range(config.num_hidden_layers)
    ]
    pooler = [torch.sum(model.bert.pooler.dense.weight != 0).item()]
    print("origin")
    j = 0
    print(j)
    results = test_f1(model, test_dataloader, model_config, False)
    pp(results["details"][f"{i}"])

    print("Start Positive CI sparse")

    for batch in positive_samples1:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t1 = w.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    print("Start Positive CI after sparse")

    for batch in positive_samples2:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t1 = ci1.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    print("Start Negative TI")

    for batch in negative_samples:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t = ti1.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    for num in range(config.num_hidden_layers):
        print(f"f({ff1[num]})")
        print("\n")
    for num in range(config.num_hidden_layers):
        print(f"f({ff2[num]})")
        print("\n")
    print(f"f({pooler})")
    print("\n")

#Module 7 in progress....
origin
0
{'precision': 0.5195530726256983,
 'recall': 0.6158940397350994,
 'f1-score': 0.5636363636363636,
 'support': 151.0}
Start Positive CI sparse
1
{'precision': 0.506578947368421,
 'recall': 0.5099337748344371,
 'f1-score': 0.5082508250825083,
 'support': 151.0}
2
{'precision': 0.5365853658536586,
 'recall': 0.5827814569536424,
 'f1-score': 0.5587301587301587,
 'support': 151.0}
3
{'precision': 0.532051282051282,
 'recall': 0.5496688741721855,
 'f1-score': 0.5407166123778502,
 'support': 151.0}
4
{'precision': 0.5286624203821656,
 'recall': 0.5496688741721855,
 'f1-score': 0.538961038961039,
 'support': 151.0}
5
{'precision': 0.535483870967742,
 'recall': 0.5496688741721855,
 'f1-score': 0.542483660130719,
 'support': 151.0}
Start Positive CI after sparse
6
{'precision': 0.6197183098591549,
 'recall': 0.5827814569536424,
 'f1-score': 0.6006825938566552,
 'support': 151.0}
7
{'precision': 0.6698113207547169,
 'recall': 0.47019867549668876,
 'f1-score': 0.

In [ ]:
for k in range(3):
    i = 12
    print("#Module " + str(i) + " in progress....")
    num_samples = 64

    positive_samples1 = sampling_class(
        train_dataloader, i, 20, num_labels, True, 4, device=device
    )
    positive_samples2 = sampling_class(
        train_dataloader, i, num_samples, num_labels, True, 4, device=device
    )
    negative_samples = sampling_class(
        train_dataloader, i, num_samples, num_labels, False, 4, device=device
    )

    module1 = copy.deepcopy(model)
    w = WeightRemoverBert(model, p=0.9)
    ci1 = ConcernIdentificationBert(model, p=0.6)
    ti1 = TanglingIdentification(model, p=0.9)

    ff1 = [
        [
            torch.sum(
                model.bert.encoder.layer[num].intermediate.dense.weight != 0
            ).item()
        ]
        for num in range(config.num_hidden_layers)
    ]
    ff2 = [
        [torch.sum(model.bert.encoder.layer[num].output.dense.weight != 0).item()]
        for num in range(config.num_hidden_layers)
    ]
    pooler = [torch.sum(model.bert.pooler.dense.weight != 0).item()]
    print("origin")
    j = 0
    print(j)
    results = test_f1(model, test_dataloader, model_config, False)
    pp(results["details"][f"{i}"])

    print("Start Positive CI sparse")

    for batch in positive_samples1:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t1 = w.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    print("Start Positive CI after sparse")

    for batch in positive_samples2:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t1 = ci1.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    print("Start Negative TI")

    for batch in negative_samples:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t = ti1.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    for num in range(config.num_hidden_layers):
        print(f"f({ff1[num]})")
        print("\n")
    for num in range(config.num_hidden_layers):
        print(f"f({ff2[num]})")
        print("\n")
    print(f"f({pooler})")
    print("\n")

#Module 12 in progress....
origin
0
{'precision': 0.8181818181818182,
 'recall': 0.7285714285714285,
 'f1-score': 0.7707808564231738,
 'support': 210.0}
Start Positive CI sparse
1
{'precision': 0.7591623036649214,
 'recall': 0.6904761904761905,
 'f1-score': 0.7231920199501247,
 'support': 210.0}
2
{'precision': 0.7595628415300546,
 'recall': 0.6619047619047619,
 'f1-score': 0.7073791348600509,
 'support': 210.0}
3
{'precision': 0.7608695652173914,
 'recall': 0.6666666666666666,
 'f1-score': 0.7106598984771574,
 'support': 210.0}
4
{'precision': 0.7611111111111111,
 'recall': 0.6523809523809524,
 'f1-score': 0.7025641025641026,
 'support': 210.0}
5
{'precision': 0.7569060773480663,
 'recall': 0.6523809523809524,
 'f1-score': 0.7007672634271099,
 'support': 210.0}
Start Positive CI after sparse
6
{'precision': 0.7419354838709677,
 'recall': 0.7666666666666667,
 'f1-score': 0.7540983606557377,
 'support': 210.0}
7
{'precision': 0.7688442211055276,
 'recall': 0.7285714285714285,
 'f1-score